# CH14 可迭代对象、迭代器和生成器

所有生成器都是迭代器，因为生成器完全实现了迭代器接口。不过，根据《设计模式：可复用面向对象软件的基础》一书的定义，迭代器用于从集合中取出元素；而生成器用于“凭空”生成元素。

在 Python 中，所有集合都可以迭代。在 Python 语言内部，迭代器用于支持：
- for 循环
- 元组拆包
- 调用函数时，使用 * 拆包实参
- 构建和扩展集合类型
- 逐行遍历文本文件
- 列表推导、字典推导和集合推导

下面分别用几种方法构建可迭代的单词序列。

In [11]:
import re
import reprlib

In [12]:
RE_WORD = re.compile('\w+')

## 使用`__getitem__`实现

In [3]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

In [4]:
s = Sentence('I love you')
s

Sentence('I love you')

In [5]:
for word in s:
    print(word)

I
love
you


In [6]:
list(s)

['I', 'love', 'you']

### 序列可以迭代的原因： `iter` 函数  

解释器需要迭代对象 `x` 时，会自动调用 `iter(x)`。  

内置的 `iter` 函数有以下作用。  
1. 检查对象是否实现了 `__iter__` 方法，如果实现了就调用它，获取一个迭代器。  
2. 如果没有实现 `__iter__` 方法，但是实现了 `__getitem__` 方法， Python 会创建一个迭代器，尝试按顺序（从索引 0 开始）获取元素。  
3. 如果尝试失败， Python 抛出 TypeError 异常。

任何 Python 序列都可迭代的原因是，它们都实现了 `__getitem__` 方法。其实，标准的序列也都实现了 `__iter__` 方法，因此你也应该这么做。之所以对 `__getitem__` 方法做特殊处理，是为了向后兼容，而未来可能不会再这么做。

从 Python 3.4 开始，检查对象 x 能否迭代，最准确的方法是：调用 `iter(x)` 函数，如果不可迭代，再处理 TypeError 异常。这比 `isinstance(x, abc.Iterable)` 更准确，因为 `iter(x)` 函数会考虑到遗留的 `__getitem__` 方法，而 `abc.Iterable` 类则不考虑。

### 可迭代的对象与迭代器的对比

**可迭代的对象**  
使用 `iter` 内置函数可以获取迭代器的对象。如果对象实现了能返回迭代器的 `__iter__` 方法，那么对象就是可迭代的。序列都可以迭代；实现了  `__getitem__` 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

**迭代器**  
迭代器是这样的对象：实现了无参数的 `__next__` 方法，返回序列中的下一个元素；如果没有元素了，那么抛出 StopIteration 异常。 Python 中的迭代器还实现了 `__iter__` 方法，因此迭代器也可以迭代。

**标准的迭代器接口有两个方法**  
1. `__next__`  
　　返回下一个可用的元素，如果没有元素了，抛出 StopIteration 异常。
2. `__iter__`  
　　返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中

我们要明确可迭代的对象和迭代器之间的关系： Python 从可迭代的对象中获取迭代器。

下面是 abc.Iterator 类的源码

In [20]:
from collections.abc import Iterable
from abc import abstractmethod

In [22]:
class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self  # 返回自身.

    @classmethod
    def __subclasshook__(cls, C):  # 检查子类. 这里检查是否实现了 __iter__ 和 __next__ 方法. 没有考虑到 __getitem__ 但 iter 函数考虑到了。
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented

检查对象 x 是否为迭代器最好的方式是调用 `isinstance(x, abc.Iterator)`。得益于 `Iterator.__subclasshook__` 方法，即使对象 x 所属的类不是 `Iterator` 类的真实子类或虚拟子类，也能这样检查。

## 使用典型的迭代器实现

In [13]:
# 可迭代对象
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
# 迭代器    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

构建可迭代的对象和迭代器时经常会出现错误，原因是混淆了二者。要知道，可迭代的对象有个 `__iter__` 方法，每次都实例化一个新的迭代器；而迭代器要实现 `__next__` 方法，返回单个元素，此外还要实现 `__iter__` 方法，返回迭代器本身。 

因此，迭代器可以迭代，但是可迭代的对象不是迭代器。  

除了 `__iter__` 方法之外，你可能还想在 Sentence 类中实现 `__next__` 方法，让 Sentence 实例既是可迭代的对象，也是自身的迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的 Alex Martelli 所说，这也是常见的反模式。

《设计模式：可复用面向对象软件的基础》一书讲解迭代器设计模式时，在“适用性”一节中说，迭代器模式可用来：
- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，而且各个迭代器要能维护自身的内部状态，因此这一模式正确的实现方式是，每次调用 `iter(my_iterable)` 都新建一个独立的迭代器。这就是为什么这个示例需要定义 SentenceIterator 类。

可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象必须实现 `__iter__` 方法，但不能实现 `__next__` 方法。

另一方面，迭代器应该一直可以迭代。迭代器的 `__iter__` 方法应该返回自身。

## 使用生成器函数实现

实现相同功能，但却符合 Python 习惯的方式是，用生成器函数代替 SentenceIterator 类。

In [8]:
class Sentense:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.words)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

只要 Python 函数的定义体中有 `yield` 关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成器传给 `next(...)` 函数时，生成器函数会向前，执行函数定义体中的下一个 `yield` 语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出 StopIteration 异常——这一点与迭代器协议一致。

## 惰性实现

In [9]:
class Sentense:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 生成器表达式实现

In [23]:
class Sentense:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

与上例唯一的区别是 `__iter__` 方法，这里不是生成器函数了（没有 yield），而是使用生成器表达式构建生成器，然后将其返回。不过，最终的效果一样：调用 `__iter__` 方法会得到一个生成器对象。